In [3]:
import math
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# required for interactive plotting
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import numpy.polynomial as np_poly

from IPython.display import Math
from IPython.display import Latex

initialization  
$ \newcommand{\E}[1]{\mathbb{E}\left[#1\right]}$  
$ \newcommand{\V}[1]{\mathbb{V}\left[#1\right]}$
$ \newcommand{\EXP}[1]{\exp\left(#1\right)}$  
$ \newcommand{\P}{\mathbb{P}}$

**TODO**  
1. Use different basis models for the same problem
1. Compare w/ and w/o regularization
1. Use different regularizers for the same problem
1. Bias Variance Decomposition + Experiments

**Questions**
1. Equation \ref{eq:ptw} under ML and Least Squares
1. Why does Lasso act as a feature selector?


Introduction
=======

Simple Linear Model
------------------

y(\textbf{x},**w**) $ = w_0 + \sum_{i=1}^{D} w_i x_i$ 

where  
**x** $ = (x_1, \cdots, x_D)^T$

Basis Functions
---------------

$
y(\mathbf{x},\mathbf{w}) = w_0 + \sum_{j=1}^{M-1} w_i ~ \phi_j(\mathbf{x})
$ 

* where $\phi_j(\mathbf{x})$ are called basis functions. 
* Total #parameters = M

If $\phi_0(\mathbf{x}) = 1$, then  
$y(\mathbf{x},\mathbf{w})  = \sum_{j=0}^{M-1} w_i ~ \phi_j(\text{x}) = \mathbf{w}^T \phi(\mathbf{x})$ 

* Linear in **w**

Choices of Basis Functions
-------------------------

1. Polynomial Basis
  * $\phi_j(\mathbf{x}) = x^j$
  * Limitation: Global models
  * Cure: Spline Functions: fit different polynomials based on region [EOSL Hastie]
1. Gaussian Basis FUnction:
  * $\phi_j(\mathbf{x}) = \EXP{-\frac{(x-\mu_j)^2}{2s^2}}$
  * Need not be a pdf
1. Sigmoidal
  * $\phi_j(\mathbf{x}) = \sigma \left( \frac{x-\mu_j}{s} \right)$
  * where $\sigma(a) = \frac{1}{1+\EXP{-a}}$ is the logistic sigmoid function
  * Can use $\tanh$, since $\tanh(a) = 2\sigma(2a) - 1$

Loss Functions for Regression
=================

[PRML]1.5.5

Say we fit a function y(x) to give the target variable t. 
Let $\mathcal{L}(t, y(x))$ denote the loss function.

Then, Average or Expected Loss is given by,
$\E{\mathcal{L}} = \iint \mathcal{L}(t, y(x)) ~p(x,t) ~dx ~dt$

If we consider the squared loss function $\mathcal{L}(t, y(\mathbf{x})) = \left( t - y(\mathbf{x})\right)^2$, then,

\begin{array}{llr}
\E{\mathcal{L}} 
&= \iint \left( t - y(\mathbf{x})\right)^2 ~p(\mathbf{x},t) ~d\mathbf{x} ~dt
\\
\frac{\partial \E{\mathcal{L}}}{\partial y(\mathbf{x})}
&=
2 \int \left( t - y(\mathbf{x})\right) ~p(\mathbf{x},t) ~dt = 0
\\
\int t ~p(\mathbf{x},t) ~dt  
&=
\int y(\mathbf{x}) ~p(\mathbf{x},t) ~dt = y(\mathbf{x}) \int  ~p(\mathbf{x},t) ~dt
\\
\int t ~p(\mathbf{x},t) ~dt  
&=
y(\mathbf{x}) p(\mathbf{x})
\\
y(\mathbf{x})
&= 
\frac{\int t ~p(\mathbf{x},t) ~dt  }{p(\mathbf{x})}
= \int t ~p(t \mid \mathbf{x}) ~dt
\\
y(\mathbf{x}) 
&= \mathbb{E}_{t} \left[ t \mid \mathbf{x} \right]
\end{array}

Alt. Derivation   

$t = \E{\E{t \mid \mathbf{x}}}$
hence, $\E{\E{t \mid \mathbf{x}} - t}^2 = \V{t \mid \mathbf{x}}$ 

\begin{array}{llr}
\left( y(\mathbf{x}) - t \right)^2
&=
\left( y(\mathbf{x}) - \E{t \mid X} + \E{t \mid \mathbf{x}} - t \right)^2
\\
&=
\left(
    y(\mathbf{x}) - \E{t \mid \mathbf{x}} 
\right)^2 
+
2
\left(
    y(\mathbf{x}) - \E{t \mid \mathbf{x}}
\right)
\left(
    \E{t \mid \mathbf{x}} - t
\right)
+
\left(
    \E{t \mid \mathbf{x}} - t
\right)^2\\
\end{array}

Substituting into $\E{\mathcal{L}}$, the cross term vanishes (the first term of the cross term). Hence  
$$
\E{\mathcal{L}} =
\underbrace{\int
\left(
    y(\mathbf{x}) - \E{t \mid \mathbf{x}} 
\right)^2 ~p(\mathbf{x}) ~d\mathbf{x}
}_{\text{Bias Term}}
-
\underbrace{
\int
\V{t \mid \mathbf{x}} ~p(\mathbf{x}) ~d\mathbf{x}
}_{\text{Variance term}}
$$

ML and Least Squares
===========

Let *t* be given by a deterministic *y* and additive Gaussian noise as,
$t = y(\mathbf{x},\mathbf{w}) + \epsilon$ where $\epsilon \sim \mathcal{N}(0, \beta^{-1})$  
Then,  
$$
p(t \mid \mathbf{x}, \mathbf{w}, \beta) = \mathcal{N}(t \mid y(\mathbf{x},\mathbf{w}), \beta^{-1})
\label{eq:ptw}
$$
[?]  
If we assume a squared loss function, optimal t is given by conditional mean of t. In case of a Gaussian, it becomes
$\E{t \mid \mathbf{x}} = \int t ~p(t \mid \mathbf{x}) ~dt = y(\mathbf{x},\mathbf{w})$

if $\mathbf{x}$ is IID Normal given by $\ref{eq:ptw}$, then  
$$
p(\mathbf{t} \mid \mathbf{X}, \mathbf{w}, \beta)
=
\prod_{n=1}^{N}
\mathcal{N}
    \left(
        t_n \mid \mathbf{w}^T \mathbf{\phi}(\mathbf{x_n}), \beta^{-1}
    \right)
\label{eq:ptiid}
$$

Taking the log likehood, we get  
\begin{array}{ll}
\ln p(\mathbf{t} \mid \mathbf{w}, \beta)
&=
\frac{N}{2} \ln \beta
-
\frac{N}{2} \ln(2\pi)
-
\frac{\beta}{2}
\sum_{n=1}^{N}
\left(
    t_n - \mathbf{w}^T \mathbf{\phi}(\mathbf{x}_n)
\right)^2
\end{array}

Parameter Estimation
-------------------
Grad of the log likehood (wrt $\mathbf{w}$) gives,  
\begin{array}{ll}
\nabla \ln p(\mathbf{t} | \mathbf{w}, \beta)
&=
\beta
\sum_{n=1}^{N}
(t_n - \mathbf{w}^T \mathbf{\phi}(\mathbf{x}_n)) \mathbf{\phi}(\mathbf{x}_n)^T
= 0
\\
\sum_{n=1}^{N}
t_n \mathbf{\phi}(\mathbf{x}_n)^T
-
\mathbf{w}^T 
\sum_{n=1}^{N}
\mathbf{\phi}(\mathbf{x}_n) \mathbf{\phi}(\mathbf{x}_n)^T
&= 0
\end{array}

$$
\mathbf{\phi}
=
\left[
    \begin{matrix}
        \phi_1\\
        \phi_2\\
        \vdots\\
        \phi_{M-1}
    \end{matrix}
\right]
\hspace{5pt}
\mathbf{t}
=
\left[
    \begin{matrix}
        t_1\\
        t_2\\
        \vdots\\
        t_{N}
    \end{matrix}
\right]
\hspace{20pt}
\mathbf{\phi} \mathbf{\phi}^T
=
\left[
\begin{matrix}
\phi_0 \phi_0 & \phi_0 \phi_1 & \cdots & \phi_0 \phi_{M-1}\\
\phi_1 \phi_0 & \phi_1 \phi_1 & \cdots & \phi_1 \phi_{M-1}\\
\vdots        & \vdots        & \ddots       & \vdots\\
\phi_{M-1} \phi_0 & \phi_{M-1} \phi_1 & \cdots & \phi_{M-1} \phi_{M-1}\\
\end{matrix}
\right]
$$

If we let
$$
\Phi(\mathbf{X})
=
\left[
\begin{matrix}
\phi_0(x_1) & \phi_1(x_1) & \cdots & \phi_{M-1}(x_1)\\
\phi_0(x_2) & \phi_1(x_2) & \cdots & \phi_{M-1}(x_2)\\
\vdots      & \vdots      & \ddots & \vdots\\
\phi_0(x_N) & \phi_1(x_N) & \cdots & \phi_{M-1}(x_N)\\
\end{matrix}
\right]
$$

$$
\sum_{n=1}^{N}
\mathbf{\phi}(\mathbf{x}_n) ~ \mathbf{\phi}(\mathbf{x}_n)^T
=
\Phi(\mathbf{X})^T \Phi(\mathbf{X})
\\
\sum_{n=1}^{N}
t_n \mathbf{\phi}(\mathbf{x}_n)
=
\Phi(\mathbf{X})^T \mathbf{t}
$$

Hence
$
\mathbf{w}_{ML} = \left(\Phi^T \Phi \right)^{-1} \Phi^T \mathbf{t}
$
which is called *normal equations* for least squares.  
$\Phi$ is called the *design matrix* which is $N \times M$ matrix.

Bias Parameter
---------------

Now, consider the last term and seperate out the bias parameter
$\frac{\beta}{2}
\sum_{n=1}^{N}
\left(
    t_n - w_0 - \sum_{m=1}^{M-1} w_m \phi_m(\mathbf{x}_n)
\right)^2$  
Diff wrt $w_0$ and equating it to zero, we get  

\begin{array}{ll}
0
&=
\frac{\beta}{2}
\sum_{n=1}^{N}
2(t_n - w_0 - \sum_{m=1}^{M-1} w_m \phi_m(\mathbf{x}_n))
\\
N w_0
&=
\sum_{n=1}^{N}
t_n
-
\sum_{n=1}^{N}
\sum_{m=1}^{M-1}
w_m \phi_m(\mathbf{x}_n)
\\
N w_0
&=
\sum_{n=1}^{N}
t_n
-
\sum_{m=1}^{M-1}
w_m 
\sum_{n=1}^{N}
\phi_m(\mathbf{x}_n)
\\
w_0
&=
\overline{t}_n
-
\sum_{m=1}^{M-1}
w_m
\overline{\phi}_m
\end{array}

where  
$
\overline{t}_n = \frac{1}{N} \sum_{n=1}^{N} t_n \\
\overline{\phi}_m = \frac{1}{N} \sum_{n=1}^{N} \phi_m(\mathbf{x}_n) \\
$

Thus the basis $w_0$ compensates for the difference between the average of target values
and
the weighted sum of the average of the basis functions

Model parameter
--------------

Diff log likelihood wrt $\beta$ and equating it to zero, we get  
\begin{array}{ll}
0
&=
\frac{N}{2}
\frac{1}{\beta}
-
\frac{1}{2}
\sum_{n=1}^{N}
\left(
    t_n - \mathbf{w}^T \mathbf{\phi}(\mathbf{x}_n)
\right)^2\\
\frac{1}{\beta_{ML}}
&=
\frac{1}{N}
\sum_{n=1}^{N}
\left(
    t_n - \mathbf{w}^T \mathbf{\phi}(\mathbf{x}_n)
\right)^2\\
\end{array}

Sequential Learning
===========

* Stochastic Gradient can be used to find the parameters sequentially.
* Update rule: $\mathbf{w}_{\tau+1} = \mathbf{w}_{\tau} + \eta \nabla E(\mathbf{w})$
* If squared loss function is assumed,  
  $\nabla E_D(\mathbf{w}) = 
  \left( 
      t_n - 
      \mathbf{w}_{\tau}^T \mathbf{\phi}(\mathbf{x}_n)
  \right)
  \mathbf{\phi}(\mathbf{x}_n)$
* Hence, the update rule becomes,  
  $\mathbf{w}_{\tau+1}
  = \mathbf{w}_{\tau}
    + \eta
      \left( 
          t_n - 
          \mathbf{w}_{\tau}^T \mathbf{\phi}(\mathbf{x}_n)
      \right)
      \mathbf{\phi}(\mathbf{x}_n)
  $

Regularization
========

* Form: $E_D(\mathbf{w}) + \lambda E_W(\mathbf{w})$

Quadratic Regularization
-----------------------

* $E_W(\mathbf{w}) = \frac{1}{2} \mathbf{w}^T \mathbf{w}$
* Called as *weight decay* since $\mathbf{w}$ decays to zero when $\lambda$ is high
* Called as *Parameter Shrinkage* as well.

* Total Error Function:  
\begin{array}{rlr}
E_D(\mathbf{w}) + \lambda E_W(\mathbf{w})
&=
\frac{1}{2}
\sum_{n=1}^{N}
(t_n - \mathbf{w}^T \mathbf{\phi}(\mathbf{x}_n)) \mathbf{\phi}(\mathbf{x}_n)^T
+
\frac{\lambda}{2} \mathbf{w}^T \mathbf{w}
\\
0
&=
\sum_{n=1}^{N}
t_n \mathbf{\phi}(\mathbf{x}_n)^T
-
\mathbf{w}^T 
\sum_{n=1}^{N}
\mathbf{\phi}(\mathbf{x}_n) \mathbf{\phi}(\mathbf{x}_n)^T
+ \lambda \mathbf{w}^T
& \color{gray}{\text{Diff wrt w}}
\\
0
&=
\Phi^T \mathbf{t} - \mathbf{w}^T \Phi^T \Phi + \lambda \mathbf{w}^T
\\
\left( \Phi^T \Phi - \lambda \mathcal{I} \right) \mathbf{w} 
&=
\Phi^T \mathbf{t}
\\
\mathbf{w}
&= 
\left( 
    \Phi^T \Phi - \lambda \mathcal{I}
\right)^{-1} 
\Phi^T \mathbf{t}
\end{array}

Generalized Regularizer
---------------------
* $\frac{\lambda}{2} \sum_{m=0}^{M-1} \left| \mathbf{w_j} \right|^q$
* Allows complex models to be fit on smaller data sets
* The problem of finding the right model complexity transforms to that of finding the right value for $\lambda$

Lasso
-----

* By setting $q=1$ in the generalized regularizer
* Has a tendency to set some weights to zero, thereby making it a "feature selector"

Multiple Outputs
==========

* Let
  * $\mathbf{y}(\mathbf{x},\mathbf{w})$ be K dimensional
  * W be $M \times K$ matrix of parameters
  * $\mathbf{\phi}(\mathbf{x})$ is M dimensional
  * $\mathbf{t}$ is K dimensional

\begin{array}{rlr}
p(\mathbf{t} \mid \mathbf{x}, \mathbf{W}, \beta)
&=
\mathcal{N}
\left(
\mathbf{t} \mid \mathbf{W}^T \mathbf{\phi}(\mathbf{x}),
\beta^{-1} \mathcal{I} 
\right)
\\
\ln p = 0
&=
\frac{NK}{2} \ln\left(\frac{\beta}{2\pi}\right)
-
\frac{\beta}{2}
\sum_{n=1}^{N}
\left\|
\mathbf{t}_n
-
\mathbf{W}^T \mathbf{\phi}(\mathbf{x})
\right\|^2
&
\color{gray}{\text{See: Multivariate Gaussian}}
\\
\mathbf{W}_{ML}
&=
\left(
\Phi^T \Phi
\right)^{-1}
\Phi^T \mathbf{T}
= 
\Phi^{\dagger} \mathbf{T}
\end{array}
* That is, the same $\Phi^{\dagger}$ can be used for all K output target variables

Bias Variance Decomposition
===============

* Frequentist viewpoint of the model complexity issue
* For a squared loss function, the optimal prediction is given by the conditional expectation h(x),  
  $h(\mathbf{x}) = \E{t \mid \mathbf{x}} = \int t ~p (t \mid \mathbf{x}) ~dt$
* The expected square loss can be written as,
$$
\E{\mathcal{L}}
=
\int \left(
    y(\mathbf{x}) - h(\mathbf{x})
\right)^2
~p(\mathbf{x}) ~d\mathbf{x}
+
\iint \left(
    h(\mathbf{x}) - t
\right)^2
~p(\mathbf{x}, t) ~dx dt
$$

init
$\newcommand{\xb}{\mathbf{x}}$
$\newcommand{\yx}{y(\xb; \mathcal{D})}$
$\newcommand{\hx}{h(\xb)}$
$\newcommand{\ed}[1]{\mathbb{E}_D\left[ #1 \right]}$
$\newcommand{\edyx}{\ed{\yx}}$
$\newcommand{\px}{~p(\xb)}$
$\newcommand{\dx}{~d\xb}$
$\newcommand{\pxdx}{\px \dx}$

Consider the First term $\left(y(\mathbf{x}) - h(\mathbf{x})\right)^2$.  
$\pm \edyx$, we get
$$
\left\{\yx - \edyx + \edyx - \hx\right\}^2
\\
\hspace{10pt}=
\left(
    \yx - \edyx
\right)^2
+
\left(
    \edyx - \hx
\right)^2
+
2
\left(\yx - \edyx\right)
\left(\edyx - \hx\right)
$$

Taking the expectation wrt $\mathcal{D}$, the last term vanishes.  
$$
\ed{\left(\yx - \hx\right)^2}
\\
=
\underbrace{\left(
    \ed{\yx - \hx}
\right)^2}_{(\text{bias})^2}
+
\underbrace{
\ed{
 \left(
     \yx - \edyx
 \right)
}}_{\text{variance}}
$$

1. Squared-bias: Deviation of the average prediction from the desired
1. Variation of the individual predictions about the average prediction

Thus,  
\begin{array}{ll}
\text{expected loss} &= (\text{bias})^2 + \text{variance} + \text{noise}\\
&\color{green}{\text{where}}&\\
(\text{bias})^2
&=
\int \left(\edyx - \hx\right)^2 \pxdx
\\
\text{variance}
&=
\int \ed{\left( \yx - \edyx \right)^2} \pxdx
\\
\text{noise}
&=
\iint \left(\hx - t\right)^2 ~p(\xb,t) \dx ~dt
\end{array}

todo: experiments

**Practical Value**  
* Zilch
* we have only a single data set
* If there are large no. of data sets, we can combine them into a single large dataset
* This would reduce the overall level of overfitting for a given model complexity